In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('../data/LoanStats2014.csv', header=1, skip_footer=2)

C:\Users\Daniel\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


## Clean Data

In [15]:
n_cols = df.shape[1]

### Turn into correct data type

In [21]:
df.select_dtypes(exclude=[np.number]).T

,0,1,2,3,4,5,6,7,8,9,...,235619,235620,235621,235622,235623,235624,235625,235626,235627,235628
term,36 months,60 months,36 months,36 months,60 months,60 months,36 months,36 months,36 months,36 months,...,36 months,60 months,36 months,36 months,36 months,60 months,60 months,36 months,60 months,36 months
int_rate,6.99%,12.39%,13.66%,13.66%,17.14%,15.59%,13.66%,14.31%,11.99%,17.86%,...,15.61%,18.25%,7.90%,18.25%,13.53%,14.47%,19.97%,7.90%,16.99%,19.20%
grade,A,C,C,C,D,D,C,C,B,D,...,C,D,A,D,B,C,D,A,D,D
sub_grade,A3,C1,C3,C3,D4,D1,C3,C4,B5,D5,...,C4,D3,A4,D3,B5,C2,D5,A4,D1,D3
emp_title,Truck Driver Delivery Personel,MANAGEMENT,Admin Specialist,Technical Specialist,Senior Sales Professional,Programming Analysis Supervisor,Deputy sheriff,Teacher,Manufacturing Engineer,Sales,...,manager,MANAGER,sales rep,MecÃ¡nica,Director of Operations,Financial Advisor,Chief of Interpretation (Park Ranger),Server Engineer Lead,patrol,NaN
emp_length,8 years,10+ years,10+ years,< 1 year,10+ years,6 years,10+ years,10+ years,< 1 year,10+ years,...,7 years,5 years,3 years,10+ years,4 years,4 years,10+ years,3 years,7 years,10+ years
home_ownership,MORTGAGE,RENT,RENT,RENT,MORTGAGE,RENT,MORTGAGE,RENT,MORTGAGE,RENT,...,RENT,MORTGAGE,RENT,RENT,RENT,MORTGAGE,MORTGAGE,OWN,MORTGAGE,MORTGAGE
verification_status,Not Verified,Source Verified,Source Verified,Source Verified,Verified,Source Verified,Verified,Source Verified,Source Verified,Source Verified,...,Verified,Verified,Verified,Source Verified,Source Verified,Source Verified,Verified,Verified,Verified,Verified
issue_d,Dec-2014,Dec-2014,Dec-2014,Dec-2014,Dec-2014,Dec-2014,Dec-2014,Dec-2014,Dec-2014,Dec-2014,...,Jan-2014,Jan-2014,Jan-2014,Jan-2014,Jan-2014,Jan-2014,Jan-2014,Jan-2014,Jan-2014,Jan-2014
loan_status,Late (31-120 days),Fully Paid,Fully Paid,Charged Off,Current,Fully Paid,Fully Paid,Charged Off,Fully Paid,Charged Off,...,Fully Paid,Charged Off,Charged Off,Fully Paid,Current,Current,Charged Off,Fully Paid,Current,Current


In [20]:
for i in df.select_dtypes(exclude=[np.number]).columns:
    print ", '{0}'".format(i)

, 'term'
, 'int_rate'
, 'grade'
, 'sub_grade'
, 'emp_title'
, 'emp_length'
, 'home_ownership'
, 'verification_status'
, 'issue_d'
, 'loan_status'
, 'pymnt_plan'
, 'url'
, 'desc'
, 'purpose'
, 'title'
, 'zip_code'
, 'addr_state'
, 'earliest_cr_line'
, 'revol_util'
, 'initial_list_status'
, 'last_pymnt_d'
, 'next_pymnt_d'
, 'last_credit_pull_d'
, 'application_type'


In [24]:
obj_num_cols = [
        'term'
        , 'int_rate'
        , 'emp_length'
        , 'revol_util'
        ]

In [25]:
obj_date_cols = [
    'issue_d'
    , 'earliest_cr_line'
    , 'last_pymnt_d'
    , 'next_pymnt_d'
    , 'last_credit_pull_d'
    ]

In [26]:
drop_cols = [
        'url'
        , 'application_type'
        , 'policy_code'
    ]

In [30]:
df.select_dtypes(include=[np.number]).T

,0,1,2,3,4,5,6,7,8,9,...,235619,235620,235621,235622,235623,235624,235625,235626,235627,235628
id,36805548.00,38098114.00,37822187.00,37662224.0000,37612354.00,37842129.00,37682226.000000,36804663.0000,37741884.00,37800722.000,...,9715141.000000,9695736.000,9007579.00,9745590.00,9745484.00,8598660.00,9684700.00,9604874.000000,9584776.00,9199665.00
member_id,39558264.00,40860827.00,40585251.00,40425321.0000,40375473.00,40605224.00,40455307.000000,39557379.0000,40504905.00,40563521.000,...,11567259.000000,11547808.000,10799568.00,11597671.00,11597572.00,1440975.00,11536848.00,11457002.000000,11436914.00,11061576.00
loan_amnt,10400.00,15000.00,9600.00,7650.0000,12800.00,21425.00,17000.000000,23325.0000,2500.00,12975.000,...,6000.000000,8525.000,18000.00,10000.00,15000.00,18400.00,22000.00,2000.000000,20700.00,10000.00
funded_amnt,10400.00,15000.00,9600.00,7650.0000,12800.00,21425.00,17000.000000,23325.0000,2500.00,12975.000,...,6000.000000,8525.000,18000.00,10000.00,15000.00,18400.00,22000.00,2000.000000,20700.00,10000.00
funded_amnt_inv,10400.00,15000.00,9600.00,7650.0000,12800.00,21425.00,17000.000000,23325.0000,2500.00,12975.000,...,6000.000000,8525.000,17975.00,10000.00,15000.00,18400.00,22000.00,2000.000000,20700.00,9975.00
installment,321.08,336.64,326.53,260.2000,319.08,516.36,578.220000,800.7100,83.03,468.170,...,209.790000,217.650,563.23,362.78,509.25,432.64,582.50,62.590000,514.34,367.58
annual_inc,58000.00,78000.00,69000.00,50000.0000,125000.00,63800.00,75000.000000,72000.0000,89000.00,60000.000,...,27000.000000,37536.000,90000.00,42000.00,66000.00,110000.00,78000.00,83000.000000,46000.00,46000.00
dti,14.92,12.03,25.81,34.8100,8.31,18.49,23.630000,27.0300,13.77,22.420,...,26.130000,12.280,10.33,11.57,16.22,19.85,18.45,5.390000,25.65,22.78
delinq_2yrs,0.00,0.00,0.00,0.0000,1.00,0.00,0.000000,1.0000,0.00,0.000,...,0.000000,4.000,1.00,0.00,0.00,0.00,0.00,3.000000,0.00,1.00
inq_last_6mths,2.00,0.00,0.00,1.0000,0.00,0.00,0.000000,0.0000,1.00,0.000,...,3.000000,0.000,0.00,0.00,0.00,2.00,5.00,1.000000,2.00,0.00


In [33]:
for i in df.select_dtypes(include=[np.number]).columns:
    print ", '{0}'".format(i)

, 'id'
, 'member_id'
, 'loan_amnt'
, 'funded_amnt'
, 'funded_amnt_inv'
, 'installment'
, 'annual_inc'
, 'dti'
, 'delinq_2yrs'
, 'inq_last_6mths'
, 'mths_since_last_delinq'
, 'mths_since_last_record'
, 'open_acc'
, 'pub_rec'
, 'revol_bal'
, 'total_acc'
, 'out_prncp'
, 'out_prncp_inv'
, 'total_pymnt'
, 'total_pymnt_inv'
, 'total_rec_prncp'
, 'total_rec_int'
, 'total_rec_late_fee'
, 'recoveries'
, 'collection_recovery_fee'
, 'last_pymnt_amnt'
, 'collections_12_mths_ex_med'
, 'mths_since_last_major_derog'
, 'policy_code'
, 'annual_inc_joint'
, 'dti_joint'
, 'verification_status_joint'
, 'acc_now_delinq'
, 'tot_coll_amt'
, 'tot_cur_bal'
, 'open_acc_6m'
, 'open_il_6m'
, 'open_il_12m'
, 'open_il_24m'
, 'mths_since_rcnt_il'
, 'total_bal_il'
, 'il_util'
, 'open_rv_12m'
, 'open_rv_24m'
, 'max_bal_bc'
, 'all_util'
, 'total_rev_hi_lim'
, 'inq_fi'
, 'total_cu_tl'
, 'inq_last_12m'
, 'acc_open_past_24mths'
, 'avg_cur_bal'
, 'bc_open_to_buy'
, 'bc_util'
, 'chargeoff_within_12_mths'
, 'delinq_amnt'
, '